# Expand the atomic state functions in a LS-coupled basis

In [1]:
using JAC


In atomic spectroscopy, the (standard) LSJ-notation of levels is frequently applied for classifying the low-lying level structures of atoms or ions. This generally requires a *unitary transformation* from a jjJ --> LSJ-coupled basis which critically depend on the shell-structure of the atoms. Since a lack of providing a fast and proper spectroscopic notation from relativistic structure calculations may hamper the interpretation and analysis of many (computed) results, `JAC` readily provides such transformation (though presently restricted to levels with **not more than two open non-relativistic shells**).

Let us first consider the low-lying $1s^2 2s 2p\; ^{1,3}P$ levels of beryllium-like ions, whose LSJ-decompositions quickly changes from a nearly pure LSJ-coupling for neutral beryllium as the nuclear charge increases along the isoelectronic sequence. We can see this, for instance, by considering the $^1P_1 - ^3P_1$ mixing for these ions.

Most easily, the transformation of atomic states from a jjJ-coupled into a LSJ-coupled basis is made after the Hamiltonian matrix has been diagonalized. In particular, such a jjJ --> LSJ transformation is performed if we set `LSjj.Settings(true)` in `Asf.Settings(...)`.

For this, let us first briefly recall the definion of `AsfSettings(..)`

In [2]:
? AsfSettings

search: AsfSettings



`struct  ManyElectron.AsfSettings`       ... a struct for defining the settings for the atomic state functions, i.e. the self-consistent-field (SCF)          and CI computations

```
+ generateScf          ::Bool                   ... True, if a SCF need to be generated, and false otherwise 
                                                    (frozen orbitals).
+ eeInteraction        ::AbstractEeInteraction  ... Specify the e-e interaction to be included into the SCF 
                                                    computations.
+ scField              ::AbstractScField        ... Specify the self-consistent field, for instance, 
                                                    Basics.ALField(), etc.
+ startScfFrom         ::AbstractStartOrbitals  ... Specify the orbitals to start the SCF computations
+ maxIterationsScf     ::Int64                  ... maximum number of SCF iterations
+ accuracyScf          ::Float64                ... convergence criterion for the SCF field.
+ shellSequenceScf     ::Array{Subshell,1}      ... Sequence of subshells to be optimized.
+ frozenSubshells      ::Array{Subshell,1}      ... Sequence of subshells to be kept frozen.

+ eeInteractionCI      ::AbstractEeInteraction  ... Specify the e-e interaction to be included into the 
                                                    CI computations.
+ qedModel             ::AbstractQedModel       ... model for estimating QED corrections {NoneQed(), 
                                                    QedPetersburg(), QedSydney()}.
+ jjLS                 ::LSjjSettings           ... settings to control a jj-LS transformation of atomic 
                                                    level, if requested.
+ levelSelectionCI     ::LevelSelection         ... Specifies the selected levels, if any.
```

---

`ManyElectron.AsfSettings()`  ... constructor for setting the default values.

---

`ManyElectron.AsfSettings(settings::AsfSettings;`

```
        generateScf=..,       eeInteraction=..,       scField=..,            startScfFrom=..,           
        maxIterationsScf=..,  accuracyScf=..,         shellSequenceScf=..,   frozenSubshells=..,    
        eeInteractionCI=..,   qedModel=..,            jjLS=..,               levelSelection=..,     
        printout::Bool=false)
... constructor for re-defining a settings::AsfSettings.
```


and define it as follows:

In [3]:
asfSettings = AsfSettings(AsfSettings(); generateScf=true, maxIterationsScf=20, jjLS=LSjjSettings(true))

generateScf:          true  
eeInteraction:        CoulombInteraction()  
scField:              JAC.Basics.DFSField()  
startScfFrom:         StartFromHydrogenic()  
maxIterationsScf:     20  
accuracyScf:          1.0e-6  
shellSequenceScf:     Subshell[]  
frozenSubshells:      Subshell[]  
eeInteractionCI:      CoulombInteraction()  
qedModel :            NoneQed()  
jjLS:                 true  
levelSelectionCI:     Inactive LevelSelection.  


With these settings, we can calculate the five lowest levels of neutral beryllium due to the $1s^2 (2s^2 + 2s 2p)$ configurations, and which includes the $^1P_1 - ^3P_1$ levels.

In [4]:
wa = Atomic.Computation(Atomic.Computation(), name="jj-LS level transformation", 
                        grid=JAC.Radial.Grid(true), nuclearModel=Nuclear.Model(4.); 
                        configs=[Configuration("[He] 2s^2"), Configuration("[He] 2s 2p")],  
                        asfSettings=asfSettings )
perform(wa)


... in performSCF ...
>> include Configuration: 1s_1/2^2 2s_1/2^2 
>> include Configuration: 1s_1/2^2 2s_1/2^1 2p_1/2^0 2p_3/2^1 
>> include Configuration: 1s_1/2^2 2s_1/2^1 2p_1/2^1 2p_3/2^0 
(Re-) Define a new standard subshell list.
Start SCF process with hydrogenic orbitals.
>> (Re-) Define a storage array for various B-spline matrices:
Nuclear model = Fermi nuclear model for Z = 4.0 with mass = 8.08, radius R = 2.247554858082373 fm and nuclear spin I = 0, dipole moment mu = 0.0 and quadrupole moment Q = 0.0. 
Generate hydrogenic orbital for subshell 1s_1/2 
  -----------------------------------------------------------------------------
   Index    Subshell     Energies [a.u.]    Dirac-E  [a.u.]     Delta-E / |E|    
  -----------------------------------------------------------------------------
      1      1s_1/2      -8.00170451e+00    -8.00170477e+00    +3.20928495e-08    
      2      2s_1/2      -2.00053315e+00    -2.00053275e+00    -1.98702657e-07    
      3      3s_1/2   


Iteration 1 for symmetries ... 
  1s_1/2::  en [a.u.] = -2.8065646e+00;   self-cons'cy = 4.8066e-01  [1.0000e+02 for sym-block kappa = -1]
  2s_1/2::  en [a.u.] = -7.3187269e-03;   self-cons'cy = 9.9271e-01  [1.0000e+02 for sym-block kappa = -1]
  2p_1/2::  en [a.u.] = -6.3833642e-02;   self-cons'cy = 9.3816e-01  [1.0000e+02 for sym-block kappa = 1]
  2p_3/2::  en [a.u.] = -9.0019153e-02;   self-cons'cy = 9.1386e-01  [1.0000e+02 for sym-block kappa = -2]

Iteration 2 for symmetries ... 
  1s_1/2::  en [a.u.] = -4.2109300e+00;   self-cons'cy = 2.0012e-01  [1.0428e+00 for sym-block kappa = -1]
  2s_1/2::  en [a.u.] = -3.2355264e-01;   self-cons'cy = 9.5576e-01  [1.0428e+00 for sym-block kappa = -1]
  2p_1/2::  en [a.u.] = -6.1896838e-02;   self-cons'cy = 1.5404e-02  [1.0103e+00 for sym-block kappa = 1]
  2p_3/2::  en [a.u.] = -5.6919971e-02;   self-cons'cy = 2.2526e-01  [1.0073e+00 for sym-block kappa = -2]

Iteration 3 for symmetries ... 
  1s_1/2::  en [a.u.] = -3.7858223e+00;   self-

 
  LS-expansion of selected atomic levels:
 
    Level   1      0 +   -3.96495993e+02  [eV]  has weight 1.0000e+00 of 1s^2 2s^2 
       1)   1.0000e+00   of     ^1S_0

    Level   2      0 -   -3.94678939e+02  [eV]  has weight 1.0000e+00 of 1s^2 2s^1 2p^1 
       1)   1.0000e+00   of     ^3P_0

    Level   3      1 -   -3.94678676e+02  [eV]  has weight 1.0000e+00 of 1s^2 2s^1 2p^1 
       1)   1.0000e+00   of     ^3P_1

    Level   4      2 -   -3.94678150e+02  [eV]  has weight 1.0000e+00 of 1s^2 2s^1 2p^1 
       1)   1.0000e+00   of     ^3P_2

    Level   5      1 -   -3.91236657e+02  [eV]  has weight 1.0000e+00 of 1s^2 2s^1 2p^1 
       1)   1.0000e+00   of     ^1P_1

Constants.warn():  Reset global array GBL_WARNINGS.


Apart from the successfull SCF iteration, the final part of this output shows the LS-expansion of all selected levels. While, in these simple computations, the $2s^2\; ^1S_0$ and the $2s 2p\; ^3P_{0,2}$ must be pure, configuation mixing will generally affect the two $2s 2p\; ^{1,3}P_1$ already in this single-configurations approximation. For neutral beryllium, however, this mixing is so *small* that both these levels appear basically *pure* in LSJ notation.

We can repeat analogue computations for beryllium-like argon ($Z=18$) and xenon ($Z=54$), with focus especially to the corresponding LSJ-expansions:

In [5]:
wa = Atomic.Computation(wa, nuclearModel=Nuclear.Model(18.))
perform(wa)


... in performSCF ...
>> include Configuration: 1s_1/2^2 2s_1/2^2 
>> include Configuration: 1s_1/2^2 2s_1/2^1 2p_1/2^0 2p_3/2^1 
>> include Configuration: 1s_1/2^2 2s_1/2^1 2p_1/2^1 2p_3/2^0 
(Re-) Define a new standard subshell list.
Start SCF process with hydrogenic orbitals.
>> (Re-) Define a storage array for various B-spline matrices:
Nuclear model = Fermi nuclear model for Z = 18.0 with mass = 37.62, radius R = 3.37121133160794 fm and nuclear spin I = 0, dipole moment mu = 0.0 and quadrupole moment Q = 0.0. 
Generate hydrogenic orbital for subshell 1s_1/2 
  -----------------------------------------------------------------------------
   Index    Subshell     Energies [a.u.]    Dirac-E  [a.u.]     Delta-E / |E|    
  -----------------------------------------------------------------------------
      1      1s_1/2      -1.62704538e+02    -1.62704858e+02    +1.97048708e-06    
      2      2s_1/2      -4.07203373e+01    -4.07203638e+01    +6.51814562e-07    
      3      3s_1/2  

     21     22p_3/2      -4.83394884e-02    -3.34833217e-01    +5.92670171e+00    
     22     23p_3/2      -3.95901714e-02    -3.06345708e-01    +6.73792325e+00    
     23     24p_3/2      -3.39196633e-02    -2.81344908e-01    +7.29444873e+00    
     24     25p_3/2      -3.05101500e-02    -2.59284192e-01    +7.49829291e+00    
     25     26p_3/2      +1.89677301e+01    -2.39719999e-01    +1.01263831e+00    
     26     27p_3/2      +7.27798483e+01    -2.22289371e-01    +1.00305427e+00    
     27     28p_3/2      +1.98674795e+02    -2.06692987e-01    +1.00104036e+00    
     28     29p_3/2      +4.72766177e+02    -1.92682235e-01    +1.00040756e+00    
     29     30p_3/2      +1.04280025e+03    -1.80049238e-01    +1.00017266e+00    
     30     31p_3/2      +2.17985706e+03    -1.68619102e-01    +1.00007735e+00    
     31     32p_3/2      +4.34065012e+03    -1.58243829e-01    +1.00003646e+00    
     32     33p_3/2      +8.21134717e+03    -1.48797500e-01    +1.00001812e+00    
    

In [6]:
wa = Atomic.Computation(wa, nuclearModel=Nuclear.Model(54.))
perform(wa)


... in performSCF ...
>> include Configuration: 1s_1/2^2 2s_1/2^2 
>> include Configuration: 1s_1/2^2 2s_1/2^1 2p_1/2^0 2p_3/2^1 
>> include Configuration: 1s_1/2^2 2s_1/2^1 2p_1/2^1 2p_3/2^0 
(Re-) Define a new standard subshell list.
Start SCF process with hydrogenic orbitals.
>> (Re-) Define a storage array for various B-spline matrices:
Nuclear model = Fermi nuclear model for Z = 54.0 with mass = 122.58, radius R = 4.722849091608491 fm and nuclear spin I = 0, dipole moment mu = 0.0 and quadrupole moment Q = 0.0. 
Generate hydrogenic orbital for subshell 1s_1/2 
  -----------------------------------------------------------------------------
   Index    Subshell     Energies [a.u.]    Dirac-E  [a.u.]     Delta-E / |E|    
  -----------------------------------------------------------------------------
      1      1s_1/2      -1.51935927e+03    -1.51947334e+03    +7.50773745e-05    
      2      2s_1/2      -3.83773875e+02    -3.83790162e+02    +4.24378735e-05    
      3      3s_1/2

     21     22p_3/2      -3.35754053e-01    -3.02243193e+00    +8.00192242e+00    
     22     23p_3/2      -2.45984564e-01    -2.76495290e+00    +1.02403512e+01    
     23     24p_3/2      -1.85605113e-01    -2.53902497e+00    +1.26797146e+01    
     24     25p_3/2      -1.45572485e-01    -2.33969641e+00    +1.50723807e+01    
     25     26p_3/2      -1.19360934e-01    -2.16295006e+00    +1.71210886e+01    
     26     27p_3/2      -1.02522843e-01    -2.00549943e+00    +1.85614887e+01    
     27     28p_3/2      -9.23659503e-02    -1.86463482e+00    +1.91874696e+01    
     28     29p_3/2      +1.22186250e+02    -1.73810597e+00    +1.01422505e+00    
     29     30p_3/2      +5.19095923e+02    -1.62403170e+00    +1.00312858e+00    
     30     31p_3/2      +1.42318343e+03    -1.52082955e+00    +1.00106861e+00    
     31     32p_3/2      +3.26398519e+03    -1.42716062e+00    +1.00043724e+00    
     32     33p_3/2      +6.69180615e+03    -1.34188594e+00    +1.00020053e+00    
    

From the last two computations, we see that relative contribution of the $^3P_1$ (non-relativistic LSJ) configuration state of the $2s 2p\; ^{3}P_1$ level drops from 1.00 ($Z=4$) to 0.85 ($Z=18$) to 0.76 ($Z=54$), and similarly also for the $2s 2p\; ^{1}P_1$ level. This change in non-relativistic LSJ decomposition is also the main reason why the $2s 2p\; ^{3}P_1 - 2s^2\; ^1S_0$ (intercombination) line is very weak, when compared with the $2s 2p\; ^{1}P_1 - 2s^2\; ^1S_0$ resonance line, for neutral beryllium but become comparable to each other for medium and heavy elements. The transition strength of all these transitions arise from the (resonance) decay within the singlett system, while the $2s 2p\; ^{3}P$ represents the lowest term (levels) of the triplett system.

The jjJ --> LSJ transformation is presently restricted to configuration expansions for which all non-relativistic CSF have **not more than two open non-relativistic shells**. This restriction can be released if the re-coupling coefficients are implemented for a larger number of open shells. Since these jjJ --> LSJ transformation can become quite expensive for complex shell structure, they can be further controlled by selecting individual levels (or symmetries of levels) as well as a minimum with which a (relativistic) CSF must contribute to the (selected) levels in order to be taken into account into the expansion.

In `JAC`, an improved control about such jjJ --> LSJ transformation is prepared (but not yet realized in all detail) by the parameters of the `LSjjSettings`, and is shown together with its  default settings:

In [7]:
? LSjjSettings

search: LSjjSettings



`struct  ManyElectron.LSjjSettings`  ... defines a type for the details and parameters for performing jj-LS expansions.

```
+ makeIt         ::Bool            ... True, if the jj-LS expansion is to be made and false otherwise.
+ minWeight      ::Float64         ... minimum weight with which a (relativistic) CSF must contribute to (at least one) level.
+ printWeight    ::Float64         ... minimum weight of a nonrelativistic CSF to be printed out in the final expansion.
+ levelSelection ::LevelSelection  ... Specifies the selected levels, if any.
```

---

`LSjjSettings(makeIt::Bool)`  ... constructor for the default values of jj-LS transformations.


In [8]:
LSjjSettings(true)

makeIt:            true  
minWeight:         0.05  
printWeight:       0.1  
levelSelection:    Inactive LevelSelection.  



As seen from beryllium-example above, the LS-coupling notation of levels can often be derived for a simple shell structure of atoms or ions by considering the fine-structure splitting (energies) and the total $J$ values of the atomic levels. For beryllium-like ions, for exmaple, the $^3P_{0,1,2}$ triplet can be identified quite easily by just looking for its fine-structure energies as obtained from a simple CI computations.

This identification becomes much less obvious if open $d$-shells or even $f$-shells are involved
as, for example, for the levels of $3d^3$ ground configuration of scandium-like iron. Following similar lines as above, we obtain: 


In [9]:
wa = Atomic.Computation(Atomic.Computation(), name="jj-LS level transformation", 
                        grid=JAC.Radial.Grid(true), nuclearModel=Nuclear.Model(26.); 
                        configs=[Configuration("[Ar] 3d^3")], asfSettings=asfSettings )
perform(wa)


... in performSCF ...
>> include Configuration: 1s_1/2^2 2s_1/2^2 2p_1/2^2 2p_3/2^4 3s_1/2^2 3p_1/2^2 3p_3/2^4 3d_3/2^0 3d_5/2^3 
>> include Configuration: 1s_1/2^2 2s_1/2^2 2p_1/2^2 2p_3/2^4 3s_1/2^2 3p_1/2^2 3p_3/2^4 3d_3/2^1 3d_5/2^2 
>> include Configuration: 1s_1/2^2 2s_1/2^2 2p_1/2^2 2p_3/2^4 3s_1/2^2 3p_1/2^2 3p_3/2^4 3d_3/2^2 3d_5/2^1 
>> include Configuration: 1s_1/2^2 2s_1/2^2 2p_1/2^2 2p_3/2^4 3s_1/2^2 3p_1/2^2 3p_3/2^4 3d_3/2^3 3d_5/2^0 
(Re-) Define a new standard subshell list.
Start SCF process with hydrogenic orbitals.
>> (Re-) Define a storage array for various B-spline matrices:
Nuclear model = Fermi nuclear model for Z = 26.0 with mass = 55.38, radius R = 3.756573410087755 fm and nuclear spin I = 0, dipole moment mu = 0.0 and quadrupole moment Q = 0.0. 
Generate hydrogenic orbital for subshell 1s_1/2 
  -----------------------------------------------------------------------------
   Index    Subshell     Energies [a.u.]    Dirac-E  [a.u.]     Delta-E / |E|    
  ---

  -----------------------------------------------------------------------------
   Index    Subshell     Energies [a.u.]    Dirac-E  [a.u.]     Delta-E / |E|    
  -----------------------------------------------------------------------------
      1      2p_3/2      -8.46909855e+01    -8.46909743e+01    -1.32105005e-07    
      2      3p_3/2      -3.76688688e+01    -3.76687634e+01    -2.79773042e-06    
      3      4p_3/2      -2.11887506e+01    -2.11846929e+01    -1.91502660e-04    
      4      5p_3/2      -1.36295960e+01    -1.35542226e+01    -5.53012776e-03    
      5      6p_3/2      -1.13122538e+01    -9.41010352e+00    -1.68149540e-01    
      6      7p_3/2      -9.13908230e+00    -6.91195226e+00    -2.43692962e-01    
      7      8p_3/2      -6.57908674e+00    -5.29094223e+00    -1.95793819e-01    
      8      9p_3/2      -4.77192752e+00    -4.17982022e+00    -1.24081369e-01    
      9     10p_3/2      -3.45174310e+00    -3.38519008e+00    -1.92809880e-02    
     10    

  -----------------------------------------------------------------------------
   Index    Subshell     Energies [a.u.]    Dirac-E  [a.u.]     Delta-E / |E|    
  -----------------------------------------------------------------------------
      1      3d_5/2      -3.75932034e+01    -3.75931842e+01    -5.08686757e-07    
      2      4d_5/2      -2.11542664e+01    -2.11527840e+01    -7.00737464e-05    
      3      5d_5/2      -1.35975708e+01    -1.35378838e+01    -4.38953885e-03    
      4      6d_5/2      -1.18530722e+01    -9.40064903e+00    -2.06901902e-01    
      5      7d_5/2      -9.23397647e+00    -6.90599924e+00    -2.52109937e-01    
      6      8d_5/2      -6.65226581e+00    -5.28695474e+00    -2.05240005e-01    
      7      9d_5/2      -4.88644482e+00    -4.17702006e+00    -1.45182190e-01    
      8     10d_5/2      -3.77023015e+00    -3.38314902e+00    -1.02667772e-01    
      9     11d_5/2      -3.16667331e+00    -2.79581665e+00    -1.17112384e-01    
     10    

  3d_3/2::  en [a.u.] = -3.0963829e+00;   self-cons'cy = 1.7757e-03  [8.4579e-02 for sym-block kappa = 2]
  3d_5/2::  en [a.u.] = -3.0870816e+00;   self-cons'cy = 3.0897e-05  [1.8564e-04 for sym-block kappa = -3]

Iteration 6 for symmetries ... 
  1s_1/2::  en [a.u.] = -2.5963609e+02;   self-cons'cy = 5.4913e-05  [2.4309e-03 for sym-block kappa = -1]
  2s_1/2::  en [a.u.] = -3.3207000e+01;   self-cons'cy = 3.1024e-04  [2.4309e-03 for sym-block kappa = -1]
  3s_1/2::  en [a.u.] = -6.3642689e+00;   self-cons'cy = 6.7254e-04  [2.4309e-03 for sym-block kappa = -1]
  2p_1/2::  en [a.u.] = -2.9100986e+01;   self-cons'cy = 3.1849e-04  [9.8107e-04 for sym-block kappa = 1]
  3p_1/2::  en [a.u.] = -5.1433873e+00;   self-cons'cy = 6.8969e-04  [9.8107e-04 for sym-block kappa = 1]
  2p_3/2::  en [a.u.] = -2.8646077e+01;   self-cons'cy = 1.9332e-04  [1.1930e-03 for sym-block kappa = -2]
  3p_3/2::  en [a.u.] = -5.0808696e+00;   self-cons'cy = 4.2302e-04  [1.1930e-03 for sym-block kappa = -2]
  3d_3/

Compute CI matrix of dimension 5 x 5 for the symmetry 3/2^+ ...   ... done.
Compute CI matrix of dimension 3 x 3 for the symmetry 7/2^+ ...   ... done.
Compute CI matrix of dimension 3 x 3 for the symmetry 9/2^+ ...   ... done.
Compute CI matrix of dimension 1 x 1 for the symmetry 11/2^+ ...   ... done.
Compute CI matrix of dimension 5 x 5 for the symmetry 5/2^+ ...   ... done.
Compute CI matrix of dimension 2 x 2 for the symmetry 1/2^+ ...   ... done.

  Eigenenergies:

  Level  J Parity          Hartrees                    eV                   [eV]

     1    3/2 +     -1.264960522271748e+03    -3.442132907159735e+04    -3.442132907159735e+04 
     2    5/2 +     -1.264957946212261e+03    -3.442125897344822e+04    -3.442125897344822e+04 
     3    7/2 +     -1.264954468400822e+03    -3.442116433737867e+04    -3.442116433737867e+04 
     4    9/2 +     -1.264950199699592e+03    -3.442104818010168e+04    -3.442104818010168e+04 
     5    1/2 +     -1.264856552505040e+03    -3.441849991


Apparently, there are 19 levels which belong to the single $3d^3$ configuration; although there is usually one dominant CSF in non-relativistic LSJ -coupling, the mixing of different terms can become quite sizeable (see levels 14 and 15). ...... How many levels do we see ... and what can be said about the LS-coupling of the levels in the ground configuration ?? 

Of course, the complexity increases rapidly if we consider the level structure of the potassium-like iron but with a $2p$ hole:

In [10]:
wa = Atomic.Computation(wa, configs=[Configuration("[Ne] 3s^2 3p^5 3d^2")])
perform(wa)


... in performSCF ...
>> include Configuration: 1s_1/2^2 2s_1/2^2 2p_1/2^2 2p_3/2^4 3s_1/2^2 3p_1/2^1 3p_3/2^4 3d_3/2^0 3d_5/2^2 
>> include Configuration: 1s_1/2^2 2s_1/2^2 2p_1/2^2 2p_3/2^4 3s_1/2^2 3p_1/2^2 3p_3/2^3 3d_3/2^0 3d_5/2^2 
>> include Configuration: 1s_1/2^2 2s_1/2^2 2p_1/2^2 2p_3/2^4 3s_1/2^2 3p_1/2^1 3p_3/2^4 3d_3/2^1 3d_5/2^1 
>> include Configuration: 1s_1/2^2 2s_1/2^2 2p_1/2^2 2p_3/2^4 3s_1/2^2 3p_1/2^2 3p_3/2^3 3d_3/2^1 3d_5/2^1 
>> include Configuration: 1s_1/2^2 2s_1/2^2 2p_1/2^2 2p_3/2^4 3s_1/2^2 3p_1/2^1 3p_3/2^4 3d_3/2^2 3d_5/2^0 
>> include Configuration: 1s_1/2^2 2s_1/2^2 2p_1/2^2 2p_3/2^4 3s_1/2^2 3p_1/2^2 3p_3/2^3 3d_3/2^2 3d_5/2^0 
(Re-) Define a new standard subshell list.
Start SCF process with hydrogenic orbitals.
>> (Re-) Define a storage array for various B-spline matrices:
Nuclear model = Fermi nuclear model for Z = 26.0 with mass = 55.38, radius R = 3.756573410087755 fm and nuclear spin I = 0, dipole moment mu = 0.0 and quadrupole moment Q = 0.0. 


     20     22d_3/2      -6.99313719e-02    -6.98881076e-01    +8.99381333e+00    
     21     23d_3/2      -5.73074061e-02    -6.39410177e-01    +1.01575488e+01    
     22     24d_3/2      -4.91586544e-02    -5.87219333e-01    +1.09453907e+01    
     23     25d_3/2      -4.42543582e-02    -5.41167051e-01    +1.12285595e+01    
     24     26d_3/2      +4.44126876e-01    -5.00327101e-01    +2.12654092e+00    
     25     27d_3/2      +3.24981368e+01    -4.63941576e-01    +1.01427594e+00    
     26     28d_3/2      +1.15722800e+02    -4.31385479e-01    +1.00372775e+00    
     27     29d_3/2      +3.05170350e+02    -4.02139702e-01    +1.00131775e+00    
     28     30d_3/2      +7.10349648e+02    -3.75770208e-01    +1.00052899e+00    
     29     31d_3/2      +1.53781657e+03    -3.51911831e-01    +1.00022884e+00    
     30     32d_3/2      +3.14967233e+03    -3.30255569e-01    +1.00010485e+00    
     31     33d_3/2      +6.11822047e+03    -3.10538528e-01    +1.00005076e+00    
    

  3d_3/2::  en [a.u.] = -4.9923433e+00;   self-cons'cy = 1.4502e-02  [5.3582e-02 for sym-block kappa = 2]
  3d_5/2::  en [a.u.] = -4.9740460e+00;   self-cons'cy = 1.3311e-02  [2.9165e-02 for sym-block kappa = -3]

Iteration 4 for symmetries ... 
  1s_1/2::  en [a.u.] = -2.6204331e+02;   self-cons'cy = 1.1446e-03  [1.0152e-02 for sym-block kappa = -1]
  2s_1/2::  en [a.u.] = -3.5511372e+01;   self-cons'cy = 4.1930e-03  [1.0152e-02 for sym-block kappa = -1]
  3s_1/2::  en [a.u.] = -8.2818662e+00;   self-cons'cy = 5.5999e-03  [1.0152e-02 for sym-block kappa = -1]
  2p_1/2::  en [a.u.] = -3.1397724e+01;   self-cons'cy = 2.1713e-03  [9.5032e-03 for sym-block kappa = 1]
  3p_1/2::  en [a.u.] = -7.0367376e+00;   self-cons'cy = 2.3721e-03  [9.5032e-03 for sym-block kappa = 1]
  2p_3/2::  en [a.u.] = -3.0927788e+01;   self-cons'cy = 6.7296e-05  [7.1546e-04 for sym-block kappa = -2]
  3p_3/2::  en [a.u.] = -6.9638790e+00;   self-cons'cy = 3.5448e-04  [7.1546e-04 for sym-block kappa = -2]
  3d_3/

    18    3/2 -     2.145500930130311e-01    5.838205401604493e+00    5.838205401604493e+00 
    19    7/2 -     2.261854291641612e-01    6.154819025085356e+00    6.154819025085356e+00 
    20    1/2 -     2.521311100233561e-01    6.860836962496633e+00    6.860836962496633e+00 
    21    11/2 -     2.778510416173958e-01    7.560711949510043e+00    7.560711949510043e+00 
    22    3/2 -     2.793572343862252e-01    7.601697542363191e+00    7.601697542363191e+00 
    23    7/2 -     2.889333019757032e-01    7.862275614094089e+00    7.862275614094089e+00 
    24    7/2 -     3.014159018871396e-01    8.201944458817403e+00    8.201944458817403e+00 
    25    9/2 -     3.147932853999009e-01    8.565961605520734e+00    8.565961605520734e+00 
    26    9/2 -     3.267010402521464e-01    8.889988119436714e+00    8.889988119436714e+00 
    27    5/2 -     3.468207973182871e-01    9.437474595592089e+00    9.437474595592089e+00 
    28    7/2 -     3.479327671750525e-01    9.467732836607139e+00   

    Level   34      3/2 -   -3.41386145e+04  [eV]  has weight 1.0000e+00 of 1s^2 2s^2 2p^6 3s^2 3p^5 3d^2 
       1)   3.2925e-01   of     ^2P_3/2
       2)   1.6598e-01   of     ^2D_3/2
       3)   1.5702e-01   of     ^4D_3/2

    Level   35      1/2 -   -3.41386140e+04  [eV]  has weight 1.0000e+00 of 1s^2 2s^2 2p^6 3s^2 3p^5 3d^2 
       1)   4.0954e-01   of     ^4D_1/2
       2)   2.7683e-01   of     ^4D_1/2
       3)   2.3596e-01   of     ^2P_1/2

    Level   36      9/2 -   -3.41382112e+04  [eV]  has weight 1.0000e+00 of 1s^2 2s^2 2p^6 3s^2 3p^5 3d^2 
       1)   4.7963e-01   of     ^4F_9/2
       2)   3.1754e-01   of     ^2G_9/2
       3)   1.8197e-01   of     ^4G_9/2

    Level   37      7/2 -   -3.41381310e+04  [eV]  has weight 1.0000e+00 of 1s^2 2s^2 2p^6 3s^2 3p^5 3d^2 
       1)   4.8412e-01   of     ^2F_7/2
       2)   2.7359e-01   of     ^4F_7/2
       3)   2.1222e-01   of     ^2G_7/2

    Level   38      3/2 -   -3.41351032e+04  [eV]  has weight 1.0000e+00 of 1s^2 2s^2 2p


This results in 45 fine-structure levels, and which are shown together with their corresponding LSJ expansions. For various applications, it migt indeed be desirable to better recognized the parents terms in this LSJ expansion, a feature that could be implemented quite easily.
